# Статистический анализ данных

*Приложение предназначено для проведения статистического анализа данных. Для корректной работы программы необходимо подгрузить Excel-файл с данными, которые могут содержать как численные, так и категориальные значения. Количество столбцов с параметрами неограничено.*

In [ ]:
!pip install voila
!jupyter serverextension enable --sys-prefix voila 

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import VBox
from ipywidgets.widgets.interaction import show_inline_matplotlib_plots
import scipy.stats as st
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
btn_upload = widgets.FileUpload(description = 'Загрузить')
btn_run = widgets.Button(description = 'Сводные таблицы')
btn_plot = widgets.Button(description = 'Построить графики')
btn_clean = widgets.Button(description = 'Очистить')
out_pl = widgets.Output()

In [4]:
def on_click_tables(change):
    try:
        df = pd.read_excel(btn_upload.data[-1])

        cat = [i for i in df.columns if df[i].dtype == 'O']

        out_pl.clear_output()

        with out_pl:

            print("\033[1m" + 'Значения P10:' + "\033[0m")
            display(df.groupby(cat).agg(lambda x: st.t.ppf(0.10, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))))
            print(' ')

            print("\033[1m" + 'Значения P50:' + "\033[0m")
            display(df.groupby(cat).agg(lambda x: st.t.ppf(0.50, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))))
            print(' ')

            print("\033[1m" + 'Значения P90:' + "\033[0m")
            display(df.groupby(cat).agg(lambda x: st.t.ppf(0.90, df=len(x)-1, loc=np.mean(x), scale=st.sem(x))))
            print(' ')
    except:
        out_pl.clear_output()
        with out_pl: print('Вероятно, отсутствуют данные либо подгружен некорректный файл.')


def on_click_graphs(change):
    try:
        df = pd.read_excel(btn_upload.data[-1])

        num = [i for i in df.columns if df[i].dtype != 'O']

        out_pl.clear_output()
        with out_pl:
            for i, j in enumerate(num):
                fig = plt.figure(figsize=(7,4), clear = True)
                x = df[j]
                sns.distplot(x)
                plt.axvline(st.t.ppf(0.10, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)), c='red', label='P10')
                plt.axvline(st.t.ppf(0.50, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)), c='blue', label='P50')
                plt.axvline(st.t.ppf(0.90, df=len(x)-1, loc=np.mean(x), scale=st.sem(x)), c='green', label='P90')
                plt.legend(loc = 'upper right')
            show_inline_matplotlib_plots()
    except:
        out_pl.clear_output()
        with out_pl: print('Вероятно, отсутствуют данные либо подгружен некорректный файл.')
        
def on_click_clean(change):
    out_pl.clear_output()

btn_run.on_click(on_click_tables)
btn_plot.on_click(on_click_graphs)
btn_clean.on_click(on_click_clean)

In [5]:
VBox([btn_upload, btn_run, btn_plot, btn_clean, out_pl])